In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append("../")

location = "remote"
if location == "remote":
    # TODO: hacky, shouldn't be necessary
    os.environ["WANDB_NOTEBOOK_NAME"] = "lustre_scratch/coralshift/notebooks/convlstm.ipynb"
    os.chdir("/lustre_scratch/orlando-code/coralshift/")


In [2]:
from __future__ import annotations

from pathlib import Path
import xarray as xa
import numpy as np

import wandb
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
# importing entire keras library leads to quasi-errors: https://stackoverflow.com/questions/65493824/tensorflow-gpu-memory-allocation
from tensorflow.keras import layers, models, losses, optimizers, callbacks


import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbCallback


# from sklearn import model_selection
# from sklearn.preprocessing import normalize
# from scipy.interpolate import interp2d
# from sklearn.utils import class_weight

import xbatcher

from sklearn.utils import class_weight
# import rasterio
# # from rasterio.plot import show
# import rioxarray as rio

# # from bs4 import BeautifulSoup
# # import requests


#issues with numpy deprecation in pytorch_env
from coralshift.processing import spatial_data
from coralshift.utils import file_ops, directories
from coralshift.plotting import spatial_plots, model_results
from coralshift.dataloading import data_structure, climate_data

from keras.models import Sequential
from keras.layers import Conv3D
from keras.layers import ConvLSTM2D
from keras.layers import BatchNormalization
# import keras
import numpy as np
import pylab as plt

from time import sleep
from tqdm.notebook import tqdm

# from keras.layers import Flatten

2023-06-24 08:21:31.389417: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        print(tf.config.experimental.get_visible_devices('GPU'))
    except RuntimeError as e:
        print(e)

2 Physical GPUs, 1 Logical GPUs
[PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


2023-06-24 08:21:36.360914: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-24 08:21:36.970597: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 29769 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB-LS, pci bus id: 0000:89:00.0, compute capability: 7.0


In [42]:
!nvidia-smi

Sat Jun 24 08:49:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:86:00.0 Off |                    0 |
| N/A   32C    P0    46W / 163W |  31460MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:89:00.0 Off |                    0 |
| N/A   

In [ ]:
np.unique(labels)

In [ ]:
rows, cols = 40, 60
seq_len = 10
num_fs = 5
target_frac = 0.1
cells = 32

# Creating the features array
features = np.random.random((cells, seq_len, rows, cols, num_fs)).astype(np.float32)

# Creating the label array
labels = np.random.choice((2), size=(cells, 1, rows, cols, 1), p=[1-target_frac,target_frac])

class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels.flatten())

# Verifying the shape of the arrays
print("Features shape:", features.shape)
print("Label shape:", labels.shape)
print("Class weights:", class_weights)



# # Creating the features array in form (sequence_length, width, height, num_features)
# features = np.random.random((num_samples, seq_len, rows, cols, num_fs)).astype(np.float32)

# # # Creating the label array
# # labels = np.repeat(np.expand_dims(
# #     np.random.choice((2), size=(rows, cols), p=[1-target_frac,target_frac]).reshape((1,40,60,1)), axis=0),
# #     num_samples, axis=0)

# # batches_X = np.repeat(np.expand_dims(features,axis=0), 100, axis=0)

# class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels.flatten())

# # Verifying the shape of the arrays
# print("Features shape:", features.shape)
# print("Label shape:", labels.shape)
# print("Class weights:", class_weights)



In [ ]:
# send numpy features/label array to tf.Data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((features, labels))

BATCH_SIZE = 5
SHUFFLE_BUFFER_SIZE = 10

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
print("Number of batches:", len(train_dataset))

## Replace toy with real

In [56]:
# TODO: replace with all variables
dir = directories.get_datasets_dir() / "test"
Xs, ys = np.load(dir / "Xs_conv.npy"), np.load(dir / "ys_conv.npy")

# Xs = np.expand_dims(Xs, axis=0)
# ys = np.expand_dims(np.expand_dims(np.expand_dims(ys, axis=-1), axis=-1), axis=0)

print("Xs shape: ", Xs.shape)
print("ys shape: ", ys.shape)


Xs shape:  (187, 137, 4, 336)
ys shape:  (187, 137)


In [38]:
# X_train = Xs[:1000, :, :20]
# y_train = ys[:1000]
X_train = Xs
y_train = ys


class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.flatten())
print("class weights: ", class_weights)

class weights:  [0.54231583 6.40795398]


In [39]:
# send numpy features/label array to tf.Data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset.element_spec


(TensorSpec(shape=(187, 137, 4, 336), dtype=tf.float64, name=None),
 TensorSpec(shape=(187, 137, 1, 1), dtype=tf.float64, name=None))

In [40]:
BATCH_SIZE = 12
SHUFFLE_BUFFER_SIZE = 2

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
print("Number of batches:", len(train_dataset))
train_dataset.element_spec

Number of batches: 1


(TensorSpec(shape=(None, 187, 137, 4, 336), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 187, 137, 1, 1), dtype=tf.float64, name=None))

In [34]:
# features.shape[2:]
inp = layers.Input(shape=(X_train.shape[1:]))
inp.shape

TensorShape([None, 187, 137, 4, 336])

In [65]:

def stitch_up(X, rows, cols):
    # nested concat split calls to
    # stitch back together a image
    # from a 5 x 3 grid of sub-images
    return tf.concat(
            tf.split(
                tf.concat(
                    tf.split(
                        X, cols, axis=0),
                    axis=2),
                rows,
                axis=0),
            axis=1
        )


def pad_to_box(X, rows, cols):
    height = tf.shape(X)[0]
    width = tf.shape(X)[1]
    target_height = tf.cast(tf.math.ceil(height / rows) * rows, tf.int32)
    target_width = tf.cast(tf.math.ceil(width / cols) * cols, tf.int32)

    # Pad the image to the target height and width
    return tf.image.resize_with_pad(X, 0, 0, target_height, target_width)

def 

def cut_squares(X, rows, cols):
    # nested split concat calls
    # to break image into 5 x 3 grid
    # of 448 * 448 sub-images
    return tf.concat(
        tf.split(
            tf.concat(
                tf.split(X, rows, axis=1),
                axis=0
            ),
            cols,
            axis=2),
        axis=0
    )

In [47]:
ten = tf.convert_to_tensor(Xs)

In [66]:
Xs.shape

(187, 137, 4, 336)

In [64]:
# cut_squares(ten, 5, 3)
pad_to_box(Xs, 10, 10)

<tf.Tensor: shape=(187, 190, 140, 336), dtype=float64, numpy=
array([[[[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        ...,

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
  

In [41]:
####################################
# DEFINE CONVOLUTIONAL LSTM MODEL
####################################


class ConvLSTModel(tf.keras.Model):
    """Convolutional LSTM model for binary classification of spatiotemporal data"""

    # initialise class instance to define layers of the model
    def __init__(self, filters: list[int] = [32, 64, 128], kernel_sizes: list[int] = [3,3,3], 
        strides: list[int] = [1,1,1], num_layers: int = 3, padding: str = "same", activation: str = "relu"
        # input_shape: tuple[int] = INPUT_SHAPE,
        ):
        """
        Initialize the ConvLSTM model (ConvLSTModel)

        Parameters
        ----------
            filters (list[int]): List of filters for each ConvLSTM2D layer.
            kernel_sizes (list[int]): List of kernel sizes for each ConvLSTM2D layer.
            strides (list[int]): List of strides for each ConvLSTM2D layer.
            num_layers (int): Number of ConvLSTM2D layers.
            padding (str): Padding mode for ConvLSTM2D layers.
            activation (str): Activation function to use.
        """

        # initialise CoonvLSTModel model as subclass of tf.keras.Model
        super(ConvLSTModel, self).__init__()

        # check argument consistency
        assert(len(filters) == num_layers)
        assert(len(kernel_sizes) == len(filters))
        assert(len(strides) == len(filters))

        self.conv_lstm_0 = layers.ConvLSTM2D(filters=filters[0],
                                            kernel_size=(kernel_sizes[0], kernel_sizes[0]),
                                            strides=(strides[0], strides[0]),
                                            padding=padding,
                                            return_sequences=True,
                                            activation=activation)


        self.conv_lstm_1 = layers.ConvLSTM2D(filters=filters[1],
                                            kernel_size=(kernel_sizes[1], kernel_sizes[1]),
                                            strides=(strides[1], strides[1]),
                                            padding=padding,
                                            return_sequences=True,
                                            activation=activation)


        self.conv_lstm_2 = layers.ConvLSTM2D(filters=filters[2],
                                            kernel_size=(kernel_sizes[2], kernel_sizes[2]),
                                            strides=(strides[2], strides[2]),
                                            padding=padding,
                                            return_sequences=True,
                                            activation=activation)

        self.batch_norm_0 = layers.BatchNormalization()
        self.batch_norm_1 = layers.BatchNormalization()
        self.batch_norm_2 = layers.BatchNormalization()

        # reducing parameter space to 1 to match label feature dimension
        self.dense_layer_out = layers.Dense(1)


    def call(self, inputs, training=False):
        # for now, threee convLSTM2D layers with batchnorm sandwiches
        x = self.conv_lstm_0(inputs)
        x = self.batch_norm_0(x, training=training)
        x = self.conv_lstm_1(x)
        x = self.batch_norm_1(x, training=training)
        x = self.conv_lstm_2(x)
        x = self.batch_norm_2(x, training=training)
        x = self.dense_layer_out(x)

        return x


###########################
# INITIALISE PARAMETERS
###########################

num_features = 5
seq_length = 100

# configure model
LEARNING_RATE = 1e-3    # TODO: adjustable (not particularly preessing though)
INPUT_SHAPE = (None, seq_length, None, None, num_features)
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
EPOCHS = 4
ACTIVATION = "relu"
BATCH_SIZE_PER_REPLICA = 5




wandb.init(
    project="new_conv_lstm",
    entity="orlando-code",
    settings=wandb.Settings(start_method="fork")
    )

wandb.config = {
    "learning_rate": LEARNING_RATE,
    "epochs": EPOCHS,
    "activation": ACTIVATION,
    "batch_size_per_replica": BATCH_SIZE_PER_REPLICA
}


# set up distributed processing
mirrored_strategy = tf.distribute.MirroredStrategy()
GLOBAL_BATCH_SIZE = (BATCH_SIZE_PER_REPLICA * mirrored_strategy.num_replicas_in_sync)

with mirrored_strategy.scope():
    conv_lstm = ConvLSTModel(activation=wandb.config["activation"])
    DIST_OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

    def compute_loss(
        labels: tf.Tensor, predictions: tf.Tensor, class_weights: tuple[float] = None, distributed: bool = True):
        """Compute binary cross-entropy loss between ground-truth binary labels and predicted probabilities,
        incorporating class weights.

        Parameters
        ----------
            labels (tf.Tensor): True binary labels, where 0 represents the negative class.
            predictions (tf.Tensor): Predicted labels as probability values between 0 and 1.
            class_weights (np.ndarray): Weights for each class. If None, no class weights will be applied.
            distributed (bool): Flag indicating if the computation is distributed.

        Returns
        -------
            float: Negative log likelihood loss computed using binary cross-entropy loss between 'labels' and 
                'predictions', incorporating class weights if provided.
        """
        per_example_loss = negative_log_likelihood(labels, predictions, class_weights, distributed)
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size=GLOBAL_BATCH_SIZE)

    train_accuracy = tf.keras.metrics.BinaryCrossentropy(name="train_accuracy")
    test_accuracy = tf.keras.metrics.BinaryCrossentropy(name="test_accuracy")
    # would use BinaryAccuracy if predicting ones/zeros

# cast datasets to mirrored versions for distributed processing
dist_train_dataset = mirrored_strategy.experimental_distribute_dataset(train_dataset)


#################################
# DEFINE TRAIN/TEST FUNCTIONS
#################################


def negative_log_likelihood(
    labels: np.ndarray, predictions: np.ndarray, class_weights: np.ndarray = None, distributed: bool=False) -> float:
    """Compute binary cross-entropy loss between ground-truth binary labels and predicted probabilities,
    incorporating class weights.

    Parameters
    ----------
    y (np.ndarray): true binary labels, where 0 represents the negative class
    y_pred (np.ndarray): predicted labels (as probability value between 0 and 1)
    class_weights (np.ndarray): weights for each class. If None, no class weights will be applied.

    Returns
    -------
    float: negative log likelihood loss computed using binary cross-entropy loss between 'y' and 'y_pred',
    incorporating class weights if provided
    """
    bce = tf.keras.losses.BinaryCrossentropy()  

    if distributed:
        bce = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)  

    print(tf.shape(labels))
    print(tf.shape(predictions))

    # TODO: this may be broken, throwing error associated with numpy/tensor incompatability
    if class_weights is not None:
        # sample_weights = tf.gather(class_weights, np.asarray(tf.concat(labels.values, axis=0), dtype=np.int32))
        sample_weights = tf.gather(class_weights, tf.cast(labels, dtype=tf.int32))

        # reshape to match size of y and y_pred
        return bce(labels, predictions, sample_weight=tf.reshape(sample_weights, (-1, 1)))

    return bce(labels, predictions)


@tf.function
def train_step(conv_lstm: tf.keras.Model, features: tf.Tensor, labels: tf.Tensor, 
    optimizer: tf.keras.optimizers.Optimizer = OPTIMIZER, learn_rate: float = LEARNING_RATE, 
    class_weights: tuple = None, distributed: bool=False):
    
    with tf.GradientTape() as lstm_tape:
        predictions = conv_lstm(features, training=True)
        # Calculate loss
        loss = compute_loss(labels, predictions, class_weights, distributed)
    
    # get trainable variables
    trainable_vars = conv_lstm.trainable_variables
    # Calculate gradient          `
    lstm_grads = lstm_tape.gradient(loss, trainable_vars)
    # And then apply the gradient to change the weights
    optimizer.apply_gradients(zip(lstm_grads, trainable_vars))

    train_accuracy.update_state(labels, predictions)
    return loss


def test_step(conv_lstm: tf.keras.Model, features: tf.Tensor, labels: tf.Tensor):

    predictions = conv_lstm(features, training=False)
    test_accuracy.update_state(labels, predictions)


@tf.function
def distributed_train_step(model: tf.keras.Model, features: tf.Tensor, labels: tf.Tensor, 
    optimizer: tf.keras.optimizers.Optimizer = DIST_OPTIMIZER, learn_rate: float = LEARNING_RATE,
    class_weights: tuple = None, distributed: bool = True):
    """Performs a distributed training step on the given model.

    Parameters
    ----------
        model (tf.keras.Model): The model to train.
        features (tf.Tensor): Input features for training.
        labels (tf.Tensor): True labels for training.
        optimizer (tf.keras.optimizers.Optimizer): Optimizer to use for training. Default is DIST_OPTIMIZER.
        learn_rate (float): Learning rate for the optimizer. Default is LEARNING_RATE.
        class_weights (tuple): Weights for each class. If None, no class weights will be applied. Default is None.
        distributed (bool): Flag indicating if the computation is distributed. Default is True.

    Returns
    -------
        tf.Tensor: Loss computed during the training step.
    """
    per_replica_losses = mirrored_strategy.run(train_step, args=(
        model, features, labels, optimizer, learn_rate, class_weights, distributed,))
    return mirrored_strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)


@tf.function
def distributed_test_step(conv_lstm: tf.keras.Model, features: tf.Tensor, labels: tf.Tensor):
    """Performs a distributed test step on the given model.

    Parameters
    ----------
        conv_lstm (tf.keras.Model): The model to evaluate.
        features (tf.Tensor): Input features for evaluation.
        labels (tf.Tensor): True labels for evaluation.

    Returns
    -------
        None
    """
    return mirrored_strategy.run(test_step, args=(conv_lstm, features, labels,))





################
# RUN MODEL
################

for epoch in tqdm(range(wandb.config["epochs"]), desc="epochs", position=0):
    # Just total loss
    epoch_loss = 0
    # Iterate through the train dataset
    for features, labels in tqdm(dist_train_dataset, desc=" train batches", position=1):
        loss = distributed_train_step(conv_lstm, features, labels, DIST_OPTIMIZER, wandb.config["learning_rate"],
            # class_weights
            )
        epoch_loss += loss
        wandb.log({"epoch loss": epoch_loss, "train accuracy": train_accuracy.result()})

    # Iterate through the test dataset
    for features, labels in tqdm(dist_train_dataset, desc=" test batches", position=1):
        distributed_test_step(conv_lstm, features, labels)
        wandb.log({"test accuracy": test_accuracy.result()})


wandb.finish()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


2023-06-24 08:48:26.544440: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_DOUBLE
      type: DT_DOUBLE
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 1
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\025TensorSliceDataset:29"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 187
        }
        dim {
          size: 137
        }
        dim {
          size: 4
        }
        dim {
          size: 336
        }
      }
      shape {
        dim {
          size: 187
        }
        dim {
          size: 137
        }

epochs:   0%|          | 0/4 [00:00<?, ?it/s]

 train batches: 0it [00:00, ?it/s]

Tensor("Shape:0", shape=(5,), dtype=int32, device=/job:localhost/replica:0/task:0/device:GPU:0)
Tensor("Shape_1:0", shape=(5,), dtype=int32, device=/job:localhost/replica:0/task:0/device:GPU:0)
Tensor("Shape:0", shape=(5,), dtype=int32, device=/job:localhost/replica:0/task:0/device:GPU:0)
Tensor("Shape_1:0", shape=(5,), dtype=int32, device=/job:localhost/replica:0/task:0/device:GPU:0)


...

 test batches: 0it [00:00, ?it/s]

 train batches: 0it [00:00, ?it/s]

 test batches: 0it [00:00, ?it/s]

 train batches: 0it [00:00, ?it/s]

 test batches: 0it [00:00, ?it/s]

In [ ]:
INPUT_SHAPE = layers.Input(shape=(features.shape[1:]))
INPUT_SHAPE

In [ ]:
4*10*60*40

In [ ]:
inp = layers.Input(shape=(features.shape[1:]))

convlstm = ConvLSTModel()

convlstm.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.FalseNegatives()])
# convlstm.build(INPUT_SHAPE)
# convlstm.build(features.shape[:])

convlstm.summary()

In [ ]:
inputs = tf.keras.layers.Input(shape=(3,))
outputs = tf.keras.layers.Dense(2)(inputs)
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="Adam", loss="mse", metrics=["mae"])
model.metrics_names


In [ ]:
outputs

## Save model and weights

[Source](https://towardsdatascience.com/creating-a-trashy-model-from-scratch-with-tensorflow-2-an-example-with-an-anomaly-detection-27f0d1d7bd00)

In [ ]:
# encoder.save_weights(filepath='encoder_model.tf')

# # Saving the configuration
# import json
# ## Encoder
# encoder_dict = {'fillters': [32, 64, 128, 196, 256, 1024],
#                 'kernel_sizes': [3, 3, 3, 3, 3, 3],
#                 'strides': [2, 2, 2, 2, 2, 2],
#                 'num_of_layers': 6}
# with open('encoder_config.json', 'w') as fp:
#     json.dump(encoder_dict, fp)

# ## Decoder
# decoder_dict = {'fillters': [1024, 256, 196, 128, 64, 32, 1], 
#                 'kernel_sizes': [3, 3, 3, 3, 3, 3, 1],
#                 'strides': [2, 2, 2, 2, 2, 2, 1],
#                 'num_of_layers': 7}
# with open('decoder_config.json', 'w') as fp:
#     json.dump(decoder_dict, fp)
